# Yelp API - Lab



## Introduction 

Now that we've seen how the Yelp API works, it's time to put those API and SQL skills to work in order to do some basic business analysis! Taking things a step further, you'll also independently explore how to perform pagination in order to retrieve a full results set from the Yelp API!

## Objectives

You will be able to:
* Create a DB on AWS to store information from Yelp about businesses
* Create HTTP requests to get data from Yelp API
* Parse HTTP responses and insert the information into your DB
* Perform pagination to retrieve troves of data!
* Write SQL queries to answer questions about your data 

## Problem Introduction

For this lab you will analyze the yelp data for a group of businesses to learn more about an industry. You will choose a type of business (Italian Restuarants, Nail Salons, Crossfit gyms) and a location to analyze. Then you will get data from the Yelp API, store that data in a SQL Database on AWS, and write queries to answer questions about the data. 


### Process:

1. Read through the SQL questions and the API documentation to determine which pieces of information you need to pull from the Yelp API.

2. Create a DB schema with 2 tables. One for the businesses and one for the reviews.

3. Create code to:
  - Perform a search of businesses using pagination
  - Parse the API response for specific data points
  - Insert the data into your AWS DB

4. Use the functions above in a loop that will paginate over the results to retrieve all of the results. 

5. Create functions to:
  - Retrieve the reviews data of one business
  - Parse the reviews response for specific review data
  - Insert the review data into the DB

6. Using SQL, query all of the business IDs. Using the 3 Python functions you've created, run your business IDs through a loop to get the reviews for each business and insert them into your DB.

7. Write SQL queries to answer the following questions about your data.


Bonus Steps:  
- Place your helper functions in a package so that your final notebook only has the major steps listed.
- Rewrite your business search functions to be able take an argument for the type of business you are searching for.
- Add another group of businesses to your database.


 
## SQL Questions:

- Which are the 5 most reviewed businesses?
- What is the highest rating recieved in your data set and how many businesses have that rating?
- What percentage of businesses have a rating greater than or  4.5?
- What percentage of businesses have a rating less than 3?
- What is the average rating of restaurants that have a price label of one dollar sign? Two dollar signs? Three dollar signs? 
- Return the text of the reviews for the most reviewed restaurant. 
- Return the name of the business with the most recent review. 
- Find the highest rated business and return text of the most recent review. If multiple business have the same rating, select the restaurant with the most reviews. 
- Find the lowest rated business and return text of the most recent review.  If multiple business have the same rating, select the restaurant with the least reviews. 


## Part 1 - Understanding your data and question

Lok at the question and determine what data you will need to store in your database in order to answer the questions. Start to thin about your DB schema. What tables will you want to create and what columns will you ahve for those tables. 

Look at the API documentation, and determine what fields of the API response you will match up with the columns you want in your DB tables. 


https://www.yelp.com/developers/documentation/v3/get_started

## Part 2 - Set up the DB

Now that you are familiar with the data, create your SQL queries to create the DB and the appropriate tables. 

### Foriegn Keys

For this project you will have two tables that you need to link using a foreign key. Below is some information about how to create a foriegn key.  

http://www.mysqltutorial.org/mysql-foreign-key/



### Using DB:
    
For this lab, you can either store the data on one DB or put in on both of the partners DBs. If you decide to put it on one DB, you want to make sure both partners have access to it.  To do this you want to add a user to your DB.  

[how to add a new user](https://howchoo.com/g/mtm3zdq2nzv/how-to-add-a-mysql-user-and-grant-privileges)

In [48]:
import mysql.connector 
from mysql.connector import errorcode

In [49]:
cnx = mysql.connector.connect(
    host = "database-2.ckhgdgqfsqpw.us-east-2.rds.amazonaws.com",
    user = "admin",
    passwd = "12345678"
)

In [50]:
cursor = cnx.cursor()

In [51]:
# Create a table for the Businesses
db_name = 'businesses'

In [52]:
def create_database(cursor, database):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)

In [53]:
# check to see if the database already 
try:
    cursor.execute("USE {}".format(db_name))

#if the previous line fails because there isn't a db by that name run this line

except mysql.connector.Error as err:
    print("Database {} does not exist.".format(db_name))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor, db_name)
        print("Database {} created successfully.".format(db_name))
        cnx.database = db_name
    else:
        print(err)
        exit(1)

In [63]:
# Create a table for the reviews
DB_NAME = 'businesses'

TABLES = {}
TABLES['businesses'] = ("""
CREATE TABLE businesses
    (
    business_id varchar(50),
    name varchar(50),
    rating FLOAT,
    price FLOAT,
    review_count FLOAT,
    PRIMARY KEY (business_id)
    )
    ENGINE=InnoDB
""")
TABLES['reviews'] = ("""
CREATE TABLE reviews
    (
    business_id varchar(50),
    id varchar(50),
    review_text varchar(255),
    review_date DATE,
    PRIMARY KEY (id, business_id),
    CONSTRAINT fk_businesses FOREIGN KEY (business_id)
        REFERENCES businesses (business_id) ON DELETE CASCADE
    )
    ENGINE=InnoDB
""")

In [64]:
cnx = mysql.connector.connect(
    host = "database-2.ckhgdgqfsqpw.us-east-2.rds.amazonaws.com",
    user = "admin",
    passwd = "12345678",
    database = DB_NAME
)
cursor = cnx.cursor()

In [65]:
for table_name in TABLES:
    table_description = TABLES[table_name]
    try:
        print("Creating table {}: ".format(table_name), end='')
        cursor.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")

cursor.close()
cnx.close()

Creating table businesses: OK
Creating table reviews: OK


## Part 3 - Create ETL pipeline for the business data from the API

Now that you know what data you need from the API, you want to write code that will execute a api call, parse those results and then insert the results into the DB.  

It is helpful to break this up into three different functions (*api call, parse results, and insert into DB*) and then you can write a function/script that pull the other three functions together. 

Let's first do this for the Business endpoint.

In [54]:
import requests
import json

In [55]:
# Write a function to make a call to the yelp API

## Connect to DB server on AWS
def pull_data(term, location, offset, SEARCH_LIMIT):

    url = 'https://api.yelp.com/v3/businesses/search'
    client_id = 'oYBTmIjjT0bPql5t-f-NZQ'
    api_key = 'UJvvMieWIlpnNQ3K4G0PbQGZ32QQfXL55-AYF-Z_Lmkefepo7_10l5JT7VrlW9jnpfwcxMzH2ejh3sa2MWHGvb0d-cwRBFDu2aH1KbyiX3Nyqw3NmncpQwM6xvYpXnYx'
    headers = {
            'Authorization': 'Bearer {}'.format(api_key),
        }

    url_params = {
                    'term': term.replace(' ', '+'),
                    'location': location.replace(' ', '+'),
                    'limit': SEARCH_LIMIT,
                    'offset': offset,
                    'sort_by': 'review_count'
                }

    return requests.get(url, headers = headers, params = url_params) 

In [56]:
import pandas as pd
import numpy as np

In [57]:
def parse_API_response(response):
    yelp_dict = response.json()
    tuple_list = []
    for business in yelp_dict['businesses']:
        p = 0
        if 'price' in business.keys() and len(business['price']) > 0:
            p = len(business['price'])
        else:
            p = None
        tuple_list.append((business['id'], business['name'], business['rating'], p, business['review_count']))
    return tuple_list

In [58]:
from datetime import date, datetime, timedelta

In [59]:
# Write a function to take your parsed data and insert it into the DB
def add_businesses(data):
        add_business = ("INSERT INTO businesses "
                   "(business_id, name, rating, price, review_count) "
                   "VALUES (%s, %s, %s, %s, %s)")

        cursor.executemany(add_business, data)
        cnx.commit()

In [66]:
# Write a script that combines the three functions above into a single process.
cnx = mysql.connector.connect(
            host = "database-2.ckhgdgqfsqpw.us-east-2.rds.amazonaws.com",
            user = "admin",
            passwd = "12345678",
            database = 'businesses'
        )
cursor = cnx.cursor()


In [67]:
def Insert_Business_Data(term, location, SEARCH_LIMIT):
    offset = 0
    new_data = []
    while offset < 1000:
        response = pull_data(term, location, offset = offset, SEARCH_LIMIT = SEARCH_LIMIT)
        data = parse_API_response(response)
        new_data.append(data)
        offset+=SEARCH_LIMIT
    newer_data = []
    for data in new_data:
        for d in data:
            newer_data.append(d)
    add_businesses(newer_data)

In [68]:
Insert_Business_Data(term = 'Sporting Goods', location = 'NYC', SEARCH_LIMIT = 50)

## Part 4 -  Create ETL pipeline for the restaurant review data from the API

You've done this for the Businesses, now you need to do this for reviews. You will follow the same process, but your functions will be specific to reviews.

In [69]:
cnx = mysql.connector.connect(
            host = "database-2.ckhgdgqfsqpw.us-east-2.rds.amazonaws.com",
            user = "admin",
            passwd = "12345678",
            database = 'businesses'
        )
cursor = cnx.cursor(buffered=True)

In [70]:
# write a SQL query to pull back all of the business ids 
# you will need these ids to pull back the reviews for each restaurant

cursor.execute("SELECT * FROM businesses.businesses;")
restaurant_data_tuple_list = cursor.fetchall()

In [71]:
restaurant_id_list = []
for restaurant in restaurant_data_tuple_list:
    restaurant_id_list.append(restaurant[0])

In [72]:
# write a function that takes a business id 
# and makes a call to the API for reivews
def pull_data_reviews(restaurant_id):
    url = 'https://api.yelp.com/v3/businesses/' + restaurant_id + '/reviews'
    client_id = 'oYBTmIjjT0bPql5t-f-NZQ'
    api_key = 'UJvvMieWIlpnNQ3K4G0PbQGZ32QQfXL55-AYF-Z_Lmkefepo7_10l5JT7VrlW9jnpfwcxMzH2ejh3sa2MWHGvb0d-cwRBFDu2aH1KbyiX3Nyqw3NmncpQwM6xvYpXnYx'
    headers = {
            'Authorization': 'Bearer {}'.format(api_key),
        }
    return requests.get(url, headers = headers) 

In [73]:
response = pull_data_reviews(restaurant_id_list[0])

In [74]:
yelp_dict = response.json()

In [75]:
yelp_dict

{'reviews': [{'id': 'DlA38hwhs7hjz7-07bFvsw',
   'url': 'https://www.yelp.com/biz/suit-club-new-york-2?adjust_creative=oYBTmIjjT0bPql5t-f-NZQ&hrid=DlA38hwhs7hjz7-07bFvsw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=oYBTmIjjT0bPql5t-f-NZQ',
   'text': 'Needed a quick measurement made. Super friendly and helpful. Great and convenient location!',
   'rating': 5,
   'time_created': '2019-12-10 13:39:36',
   'user': {'id': 'WFWRKOqn6McrGmkAeOTkpw',
    'profile_url': 'https://www.yelp.com/user_details?userid=WFWRKOqn6McrGmkAeOTkpw',
    'image_url': 'https://s3-media2.fl.yelpcdn.com/photo/B1Hc87unSPoZpYVx_le1CQ/o.jpg',
    'name': 'Ace L.'}},
  {'id': '4u0qkhmZx0iOrABddShzsw',
   'url': 'https://www.yelp.com/biz/suit-club-new-york-2?adjust_creative=oYBTmIjjT0bPql5t-f-NZQ&hrid=4u0qkhmZx0iOrABddShzsw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=oYBTmIjjT0bPql5t-f-NZQ',
   'text': 'Got a shirt package out in jersey at a hotel satellite locati

In [76]:
# Write a function to parse out the relevant information from the reviews
def parse_API_response_reviews(response, restaurant_id):
    yelp_dict = response.json()
    tuple_list = []
    for reviews in yelp_dict['reviews']:
        tuple_list.append((restaurant_id, reviews['id'], reviews['text'], reviews['time_created']))
    return tuple_list

In [77]:
# write a function to insert the parsed data into the reviews table
def add_reviews(data):
    add_reviews = ("INSERT INTO reviews "
                       "(business_id, id, review_text, review_date) "
                   "VALUES (%s, %s, %s, %s)")

    cursor.executemany(add_reviews, data)
    cnx.commit()

In [78]:
cnx = mysql.connector.connect(
            host = "database-2.ckhgdgqfsqpw.us-east-2.rds.amazonaws.com",
            user = "admin",
            passwd = "12345678",
            database = 'businesses'
        )
cursor = cnx.cursor(buffered=True)

In [79]:
# combine the functions above into a single script  
def Insert_Review_Data():
    restaurant_id_list = []
    for restaurant in restaurant_data_tuple_list:
        restaurant_id_list.append(restaurant[0])
    new_data = []
    for restaurant in restaurant_id_list:
        response = pull_data_reviews(restaurant)
        data = parse_API_response_reviews(response, restaurant)
        new_data.append(data)
    newer_data = []
    for data in new_data:
        for d in data:
            newer_data.append(d)
    add_reviews(newer_data)

In [80]:
Insert_Review_Data()

## Part 5 -  Write SQL queries that will answer the questions posed. 

Now that your data is in the DB, you can answer the questions. Write SQL querires that will answer the questions. **Do not use Pandas in this lab**

Which are the 5 most reviewed businesses?

What is the highest rating recieved in your data set and how many businesses have that rating?

What percentage of businesses have a rating greater than or 4.5?

What percentage of businesses have a rating less than 3?

What is the average rating of restaurants that have a price label of one dollar sign? Two dollar signs? Three dollar signs?

Return the text of the reviews for the most reviewed restaurant.

Return the name of the business with the most recent review.

Find the highest rated business and return text of the most recent review. If multiple business have the same rating, 
select the restaurant with the most reviews.

Find the lowest rated business and return text of the most recent review. If multiple business have the same rating, select the restaurant with the least reviews.

In [136]:
# create connection
cnx = mysql.connector.connect(
            host = "database-2.ckhgdgqfsqpw.us-east-2.rds.amazonaws.com",
            user = "admin",
            passwd = "12345678",
            database = 'businesses'
        )
cursor = cnx.cursor(buffered=True)

In [152]:
# execute SQL queries
cursor.execute("""
SELECT name AS Name, review_count AS "Number Of Reviews"
FROM businesses.businesses
ORDER BY review_count DESC
LIMIT 5
;
""")
my_data = pd.DataFrame(cursor.fetchall())
my_data.columns = [x[0] for x in cursor.description]
my_data

,Name,Number Of Reviews
0,Chelsea Market,2198.0
1,B & H Photo & Electronics,1788.0
2,Macy's,1622.0
3,Strand Bookstore,1528.0
4,Yankee Stadium,1363.0


What is the highest rating recieved in your data set and how many businesses have that rating?

In [153]:
cursor.execute("""
SELECT COUNT(*) AS 'Number Of Businesses', rating AS Rating
FROM businesses.businesses
GROUP BY rating
ORDER BY rating DESC
LIMIT 1
""")
my_data = pd.DataFrame(cursor.fetchall())
my_data.columns = [x[0] for x in cursor.description]
my_data

,Number Of Businesses,Rating
0,115,5.0


What percentage of businesses have a rating greater than or 4.5?

In [154]:
cursor.execute("""
SELECT
(SELECT COUNT(*)
FROM businesses.businesses
WHERE rating >= 4.5)
/
(SELECT COUNT(*)
FROM businesses.businesses) AS 'Percentage Of Businesses With Rating >= 4.5'
""")
my_data = pd.DataFrame(cursor.fetchall())
my_data.columns = [x[0] for x in cursor.description]
my_data

,Percentage Of Businesses With Rating >= 4.5
0,0.2900


What percentage of businesses have a rating less than 3?

In [155]:
cursor.execute("""
SELECT
(SELECT COUNT(*)
FROM businesses.businesses
WHERE rating < 3)
/
(SELECT COUNT(*)
FROM businesses.businesses) AS 'Percentage Of Businesses With Rating <3'
""")
my_data = pd.DataFrame(cursor.fetchall())
my_data.columns = [x[0] for x in cursor.description]
my_data

,Percentage Of Businesses With Rating <3
0,0.1500


What is the average rating of restaurants that have a price label of one dollar sign? Two dollar signs? Three dollar signs?

In [156]:
cursor.execute("""
SELECT AVG(rating) AS 'Average Rating', price AS 'Price'
FROM businesses.businesses
GROUP BY price 
""")
my_data = pd.DataFrame(cursor.fetchall())
my_data.columns = [x[0] for x in cursor.description]
my_data

,Average Rating,Price
0,3.643725,2.0
1,3.570312,3.0
2,3.481481,4.0
3,3.909326,NaN
4,3.753731,1.0


Return the text of the reviews for the most reviewed restaurant

In [164]:
cursor.execute("""
SELECT b.name AS 'Name', b.review_count AS 'Number Of Reviews', r.review_text AS 'Review Text'
FROM businesses.businesses b
JOIN businesses.reviews r
ON b.business_id = r.business_id
WHERE b.review_count = (SELECT review_count
                         FROM businesses.businesses
                         ORDER BY review_count DESC
                         LIMIT 1)
;
""")
my_data = pd.DataFrame(cursor.fetchall())
pd.set_option('display.max_colwidth', -1)
my_data.columns = [x[0] for x in cursor.description]
my_data

,Name,Number Of Reviews,Review Text
0,Chelsea Market,2198.0,"Love Chelsea Market!\n\nI've been here a few times and love it! There are so many eateries, shops and things to see. You won't leave hungry or without..."
1,Chelsea Market,2198.0,Whenever we come to New York we love visiting Chelsea Market.\n\nIt has a wonderfully eclectic choice of eateries and stores and if you visit at Christmas or...
2,Chelsea Market,2198.0,"Chelsea Market in New York is probably one of the largest and most famous and reviewed markets I've seen, and while I wasn't majorly impressed from my visit..."


Return the name of the business with the most recent review.

In [163]:
cursor.execute("""
SELECT b.name AS 'Name', r.review_date AS 'Review Date', r.review_text AS 'Review Text'
FROM businesses.businesses b
JOIN businesses.reviews r
ON b.business_id = r.business_id
ORDER BY review_date DESC
LIMIT 1
;
""")
my_data = pd.DataFrame(cursor.fetchall())
pd.set_option('display.max_colwidth', -1)
my_data.columns = [x[0] for x in cursor.description]
my_data

,Name,Review Date,Review Text
0,Target,2020-01-24,"Always busy but the cashiers are always fast so it makes up for the business, they always have what I want in stock. The prices are a little high but that's..."


Find the highest rated business and return text of the most recent review. If multiple business have the same rating, select the restaurant with the most reviews.

In [162]:
cursor.execute("""
SELECT b.name AS 'Name', 
       r.review_date AS 'Review Date', 
       r.review_text AS 'Review Text', 
       b.review_count AS 'Review Count', 
       b.rating AS Rating
FROM businesses.businesses b
JOIN businesses.reviews r
ON b.business_id = r.business_id
ORDER BY b.rating DESC,
         b.review_count DESC,
         r.review_date DESC
LIMIT 1
""")
my_data = pd.DataFrame(cursor.fetchall())
my_data.columns = [x[0] for x in cursor.description]
pd.set_option('display.max_colwidth', -1)
my_data

,Name,Review Date,Review Text,Review Count,Rating
0,Beyond Bespoke Alterations,2020-01-18,"I came here last year to get a jumpsuit hemmed and I was beyond impressed by the result! The tailor made the length just perfect, so that I could wear...",257.0,5.0


Find the lowest rated business and return text of the most recent review. If multiple business have the same rating, select the restaurant with the least reviews.

In [161]:
cursor.execute("""
SELECT b.name AS 'Name', 
       r.review_date AS 'Review Date', 
       r.review_text AS 'Review Text', 
       b.review_count AS 'Review Count', 
       b.rating AS 'Rating'
FROM businesses.businesses b
JOIN businesses.reviews r
ON b.business_id = r.business_id
ORDER BY b.rating ASC,
         b.review_count ASC,
         r.review_date DESC
LIMIT 1
;
""")
my_data = pd.DataFrame(cursor.fetchall())
my_data.columns = [x[0] for x in cursor.description]
pd.set_option('display.max_colwidth', -1)
my_data

,Name,Review Date,Review Text,Review Count,Rating
0,Dynasty Sports,2018-10-05,LAZY staff. RUDE management. \nWent in Fri after a long wk of work to buy sneakers and an outfit. I walked around 20 mins nobody asked if I needed help. I...,1.0,1.0


# Extra Reference help

###  Pagination

Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retriving 50 at a time. Processes such as these are often refered to as pagination.

Now that you have an initial response, you can examine the contents of the json container. For example, you might start with ```response.json().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. 

**Note: be mindful of the API rate limits. You can only make 5000 requests per day, and APIs can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

***Below is sample code that you can use to help you deal with the pagination parameter and bring all of the functions together.***


***Also, something might cause your code to break while it is running. You don't want to constantly repull the same data when this happens, so you should insert the data into the database as you call and parse it, not after you have all of the data***


In [ ]:
def yelp_call(url_params, api_key):
    # your code to make the yelp call
    return data

In [ ]:
def parse_results(results):
    # your code to parse the result to make them easier to insert into the DB
    return parsed_results

In [ ]:
def db_insert(cnx, cursor, parsed results):
    # your code to insert and commit the results
    # I would create the connection and cursor outside of this function and then pass it through

In [ ]:
# create a variable  to keep track of which result you are in. 
cur = 0

#set up a while loop to go through and grab the result 
while cur < num and cur < 1000:
    #set the offset parameter to be where you currently are in the results 
    url_params['offset'] = cur
    #make your API call with the new offset number
    results = yelp_call(url_params, api_key)
    
    #after you get your results you can now use your function to parse those results
    parsed_results = parse_results(results)
    
    # use your function to insert your parsed results into the db
    db_insert(parsed_results)
    #increment the counter by 50 to move on to the next results
    cur += 50